In [20]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import plotly.express as px

In [21]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSlevels_df_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']

In [22]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url, dtype=None, parse_dates=None):
    req = urllib.request.Request(url)
    req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req), dtype=dtype, parse_dates=parse_dates)

ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all", dtype=dtypes)
ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life', 'electric_quadrupole']]
ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
ground_states_df = ground_states_df.copy()
ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
ground_states_df = ground_states_df.set_index(['z', 'a'], drop=True)

In [23]:
levels_df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


levels_df.loc[levels_df['half_life'] == 'STABLE', 'half_life'] = 'inf'
levels_df['half_life'] = levels_df['half_life'].astype('Float64')

levels_df['a'] = levels_df['z'] + levels_df['n']

levels_df = levels_df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
levels_df['energy'] = levels_df['energy'] / 1000 #MeV
levels_df['ripl_shift'] = levels_df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

levels_df = levels_df.set_index(['z', 'a'], drop=True)

In [24]:
# Filter out unknown energies

levels_df = levels_df[levels_df['energy_shift'].isna()]
levels_df = levels_df.drop(['energy_shift', 'ripl_shift'], axis='columns')

In [25]:
# Keep only even-even nuclei

df = levels_df[(levels_df.index.get_level_values('z') % 2 == 0) & (levels_df.index.get_level_values('a') % 2 == 0)]

In [26]:
# Filter out uncertain jp

df = df[df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

df['j'] = df['jp'].str[:-1]
df['p'] = df['jp'].str[-1]

odd_spins = df['j'].str.contains('/')
even_spins = ~odd_spins

df.loc[odd_spins,'j_float'] = df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
df.loc[even_spins,'j_float'] = df[even_spins]['j'].astype(float)

df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
df.loc[even_spins,'j_evenness'] = (df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)

df['p_bit'] = df['p'].str.fullmatch(r'\-').astype(int)

In [27]:
# Only consider first even levels i.e. 0^+_1, 2^+_1, 4^+_1, 6^+_1, 8^+_1, ...

first_even_levels = df[(df['jp_order'] == 1) & df['j_evenness'].fillna(False) & (df['p_bit'] == 0)].copy()

# Number of quanta of quadrupole oscillation
first_even_levels['quanta'] = (first_even_levels['j_float'].astype(int) // 2)

# Remove extraenous information
first_even_levels = first_even_levels.drop(columns=['idx', 'jp', 'jp_order', 'j', 'p', 'j_float', 'j_evenness', 'p_bit'])

In [28]:
# Filter out not enough levels
first_even_levels = first_even_levels.loc[first_even_levels.groupby(level=first_even_levels.index.names).size() >= 3]

In [29]:
def get_osc_energy(n, energy_quantum):
    return n * energy_quantum

def fit(group, func):

    x = group['quanta']
    y = group['energy']

    # fit
    popt, pcov = curve_fit(func, x, y)

    # prediction
    y_pred = func(x, *popt)

    # r-squared
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    dof_res = len(x) - 1 - 1
    dof_tot = len(x) - 1
    r2 = 1 - ((ss_res/dof_res) / (ss_tot/dof_tot))

    # RSE
    rse = np.sqrt(ss_res / dof_res)

    # goodness of fit
    goodness = rse / y.std()

    # results
    results = pd.DataFrame({'energy_quantum': popt[0], 'r2': r2, 'goodness': goodness, 'quanta': x, 'energy_pred': y_pred})
    
    return  results

In [30]:
first_even_levels_groups = first_even_levels.groupby(level=df.index.names, as_index=False)

oscillator_fit = first_even_levels_groups.apply(lambda group: fit(group, get_osc_energy)).droplevel(0)

In [31]:
merged = first_even_levels.merge(oscillator_fit, how='left', on=['z','a', 'quanta'])

oscillators = merged.groupby(level=merged.index.names, as_index=True).head(1)[['symbol', 'n', 'energy_quantum', 'r2', 'goodness']]

def get_best_goodness(df, min_goodness):
    return df[df['goodness'] >= min_goodness]

def get_worst_goodness(df, max_goodness):
    return df[df['goodness'] < max_goodness]

In [32]:
mean = oscillators['goodness'].mean()

percentages = [5, 68, 95, 99.7]
quantiles = [oscillators['goodness'].quantile(percentage/100) for percentage in percentages]

quantile_counts = [(oscillators['goodness'] <= quantile).sum() for quantile in quantiles]
# If sorting as oscillators.sort_values(by='goodness', ascending=True).iloc[idx]['goodness']
# values up to and excluding idx=quantile_count[i] will have values up to an including quantiles[i]

percentages = [quantile_count / len(oscillators) * 100 for quantile_count in quantile_counts]

fig = px.histogram(oscillators, x='goodness', nbins=100, title='Distribution of Goodness of Fit For Oscillators')
fig.update_layout(
    xaxis_title='Goodness of Fit (RSE/STD)',
    yaxis_title='Count',
    width=800,
    showlegend=False
)

for percentage, quantile in zip(percentages, quantiles):
    print(f"{percentage:.1f}% have RSE/STD < {quantile:.3f}")
    fig.add_vline(x=quantile, line_dash="dash", line_color="red", annotation_text=f"{percentage:.1f}%")

fig.show()

5.0% have RSE/STD < 0.072
67.6% have RSE/STD < 0.272
95.0% have RSE/STD < 0.410
99.3% have RSE/STD < 0.786


In [33]:
get_worst_goodness(oscillators, quantiles[0]).sort_values(by='goodness', axis=0, ascending=True)

symbol   n  energy_quantum        r2  goodness
z  a                                                 
16 36       S  20        3.263940  0.999914  0.009254
42 98      Mo  56        0.774167  0.998959  0.032258
54 132     Xe  78        0.706000  0.998632  0.036982
32 70      Ge  38        1.092425  0.997262  0.052321
   72      Ge  40        0.921623  0.996071  0.062681
60 146     Nd  86        0.651448  0.995303  0.068538
46 104     Pd  58        0.794570  0.994833  0.071882

In [34]:
fig = px.scatter(get_worst_goodness(oscillators, quantiles[0]).reset_index(), 
          x='n',
          y='z', 
          color='goodness',
          title='Goodness of Fit for Lowest 5% of Oscillator Fits',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'goodness': 'Goodness of Fit (RSE/STD)'},
          color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()

In [35]:
fig = px.scatter(get_best_goodness(oscillators, quantiles[1]).reset_index(), 
          x='n',
          y='z', 
          color='goodness',
          title='Goodness of Fit for Best Oscillator Fits (Top 32%)',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'goodness': 'Goodness of Fit (RSE/STD)'},
        #   color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          color_continuous_scale='plasma',
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [36]:
fig = px.line(get_best_goodness(oscillators, quantiles[0]).reset_index(), 
                 x='a', 
                 y='goodness', 
                 line_group='z',
                 title='Goodness of Fit vs Weight for Oscillators, Grouped by Isotopes',
                 labels={'a': 'Weight (A)', 
                        'goodness': 'Goodness of Fit (RSE/STD)',
                        'z': 'Atomic Number (Z)'})

fig.show()

fig = px.line(get_best_goodness(oscillators, quantiles[0]).reset_index(), 
                 x='a', 
                 y='goodness', 
                 line_group='n',
                 title='Goodness of Fit vs Weight for Oscillators, Grouped by Isotones',
                 labels={'a': 'Weight (A)', 
                        'goodness': 'Goodness of Fit (RSE/STD)',
                        'z': 'Atomic Number (Z)'})

fig.show()

In [48]:
residuals_df = merged.copy()
residuals_df = residuals_df.reset_index()
residuals_df["name"] = residuals_df["a"].astype(str) + residuals_df["symbol"] + " " + residuals_df['goodness'].map('{:,.2f}'.format)
residuals_df["residuals"] = residuals_df["energy"] - residuals_df["energy_pred"]
residuals_df["normalized_residuals"] = residuals_df["residuals"] / residuals_df["energy_quantum"]

best_residuals = get_worst_goodness(residuals_df, 0.13)#.sort_values(by="goodness", ascending=True)
fig = px.scatter(best_residuals, x="quanta", y="normalized_residuals", facet_col="name", facet_col_wrap=6, 
                 labels={"quanta": "Phonons (n)", "normalized_residuals": "Normalized Residuals", "name": "Nuclide"},
                 height=800)
fig.update_yaxes(range=[-0.7, 0.7])
fig.show()